In [ ]:
import sys
from os import getcwd
from os.path import join
sys.path.insert(0, join(getcwd(), "../module_code"))

from data.load import load_data
from utils import load_cli_args, init_cli_args

sys.argv = [sys.argv[0]]
load_cli_args("../options.yml")
args = init_cli_args()
preprocessed_df = load_data(args)

# Patient Type
- heart
- liver
- infection

In [ ]:
myplot.subset_legend

In [ ]:
import upsetplot
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd
groups = ["heart", "liver", "infection"]
fig = plt.figure(figsize=(10, 30))
colnames =  [f"{name}_pt_indicator" for name in groups]
indicator_cols = preprocessed_df[colnames].astype(bool)
data = pd.concat([indicator_cols, preprocessed_df["recommend_crrt"]],axis=1).replace({0: "Do not recommend CRRT", 1: "Recommend CRRT"})
myplot = upsetplot.UpSet(data.set_index(colnames), intersection_plot_elements=0, show_counts=True, show_percentages=True, element_size=50)
myplot.add_stacked_bars(by="recommend_crrt", colors=cm.Set2, elements=10)
myplot.plot()

## Numerical Breakdown

In [ ]:
myplot.total

In [ ]:
from IPython.display import display
for table in [myplot.totals, myplot.intersections]:
    display(pd.concat([table.rename("N"), (table/myplot.total*100).rename("%")], axis=1), )

In [ ]:
df = myplot._df.set_index("recommend_crrt", append=True, drop=False)
gb = df.groupby(level=list(range(df.index.nlevels)), sort=True)
counts = gb.size()
# percents = counts / df.groupby(level=list(range(df.index.nlevels - 2)), sort=True) * 100
percents = counts / myplot.intersections * 100
pd.concat([counts.rename("N"), percents.rename("%")], axis=1)